# AI Cyoda configurations Q&A with RAG Langchain

Welcome to this Jupyter notebook! This notebook serves as your guide to developing an AI-powered Question & Answer system using the Langchain library. This system utilizes the Retrieval-Augmented Generation (RAG) model that leverages OpenAI's GPT-3 model to provide intelligent and context-aware responses.

The primary purpose of this notebook is to generate Cyoda mapping configurations and resources. It does so by interacting with the data set available in the official Cyoda repository.

## What will we cover?

In this notebook, we will go through the following steps:

1. **Setting up the environment**: We will install necessary libraries and load environment variables.

2. **Initializing the AI model**: We will initialize the ChatOpenAI model with the appropriate parameters.

3. **Loading instructions and entities**: We will load instructions and entities from the official repository using the GitLoader.

4. **Splitting documents and creating a vectorstore**: We will split the loaded documents into chunks and create a vectorstore using the Chroma library.

5. **Defining prompts for contextualizing and answering questions**: We will define prompts that the AI model will use to contextualize and answer questions.

6. **Creating a retrieval chain**: We will create a retrieval chain that combines the history-aware retriever and the question-answer chain.

7. **Running the chatbot**: Finally, we will run the chatbot and see it in action!

## Let's get started!

Please follow along with the code cells and comments to understand each step of the process. If you have any questions or run into any issues, feel free to ask for help. Happy coding!

Install requirements

In [1]:
%%script echo skipping
pip install -r ../requirements.txt

skipping


### Load environment variables

In [2]:
from dotenv import load_dotenv
import os

load_dotenv()
OPENAI_API_KEY = os.environ['OPENAI_API_KEY']
WORK_DIR = os.environ['WORK_DIR']

In [3]:
%%script echo skipping
##for google colab (optional)
# This cell is optional and can be skipped
from google.colab import userdata
API_KEY = userdata.get('OPENAI_API_KEY')
WORK_DIR = userdata.get('WORK_DIR')

skipping


### Handle unsupported version of sqlite3 (optional)

In [4]:
%%script echo skipping
pip install pysqlite3-binary==0.5.2.post3

skipping


In [5]:
%%script echo skipping
import sys
__import__('pysqlite3')
sys.modules['sqlite3'] = sys.modules["pysqlite3"]

skipping


### Initialize ChatOpenAI

In [6]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_community.document_loaders import GitLoader, WebBaseLoader
from langchain_community.vectorstores import Chroma
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.chains import create_history_aware_retriever, create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.schema import HumanMessage

In [7]:
llm = ChatOpenAI(temperature=0, max_tokens = 6000, model="gpt-3.5-turbo-16k", openai_api_key=OPENAI_API_KEY)

### Load instructions and entities from the official cyoda repository

In [8]:
%%script echo skipping
loader = GitLoader(
    clone_url="https://github.com/Cyoda-platform/cyoda-ai",
    repo_path=WORK_DIR,
    branch="cyoda-ai-configurations-3.0.x",
    file_filter=lambda file_path: file_path.startswith(
        f"{WORK_DIR}/data/config-generation/mappings/"
    ),
)
docs = loader.load()
print(f"Number of documents loaded: {len(docs)}")

skipping


In [18]:
#alternatively specify only repo_path without clone_url for a local repository
loader = GitLoader(
    repo_path=WORK_DIR,
    branch="cyoda-ai-configurations-3.0.x",
    file_filter=lambda file_path: file_path.startswith(
        f"{WORK_DIR}/data/config-generation/mappings/input_scrc_paths_instruction_0.txt"
    ),
)
docs = loader.load()
print(f"Number of documents loaded: {len(docs)}")


Number of documents loaded: 0


In [16]:
print(WORK_DIR) 

/Users/kseniia/Documents/python/cyoda-ai


In [ ]:
loader = WebBaseLoader(["https://docs.oracle.com/javase/8/docs/technotes/guides/scripting/prog_guide/javascript.html"])
scripting_docs = loader.load()
docs.extend(scripting_docs)
print(f"Number of documents loaded: {len(docs)}")

### Split documents and create vectorstore

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)
vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings())
retriever = vectorstore.as_retriever(search_kwargs={"k": 10})

In [ ]:
count = vectorstore._collection.count()
print(count)

### Define prompts for contextualizing question and answering question

In [ ]:
contextualize_q_system_prompt = """Given a chat history and the latest user question \
which might reference context in the chat history, formulate a standalone question \
which can be understood without the chat history. Do NOT answer the question, \
just reformulate it if needed and otherwise return it as is."""
contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)


In [ ]:
history_aware_retriever = create_history_aware_retriever(
    llm, retriever, contextualize_q_prompt
)

### Answer question

In [ ]:
qa_system_prompt = """You are a mapping tool. You should do your best to answer the question.
Use the following pieces of retrieved context to answer the question. \

{context}"""
qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", qa_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)
question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)


### Create retrieval chain

In [ ]:
rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)

### Initialize chat history

In [ ]:
chat_history = {}

In [ ]:
# Function to add a message to the chat history
def add_to_chat_history(id, question, message):
    if id in chat_history:
        chat_history[id].extend([HumanMessage(content=question), message])
    else:
        chat_history[id] = [HumanMessage(content=question), message]

In [ ]:
# Function to clear chat history
def clear_chat_history(id):
    if id in chat_history:
        del chat_history[id]

### Define question

In [ ]:
# Function to read file content
def read_file_to_string(file_path):
    with open(file_path, 'r') as file:
        return file.read()

In [ ]:
INPUT = read_file_to_string(f"{WORK_DIR}/data/entities/tender_entity/resources/data_source_inputs/tender_input_1.json")
ENTITY = "net.cyoda.saas.model.TenderEntity"
RETURN_STRING = "Return only DataMappingConfigDto json."
question = f"Produce a mapping from this input to this target entity. Input: {INPUT}. Entity: {ENTITY}. {RETURN_STRING}. Work only with JSON attributes that are simple and not arrays. Do NOT use any atributes that are not present in schema. {RETURN_STRING}."

In [ ]:
import uuid

# Generate a unique ID for the chat session
id = uuid.uuid1()

In [ ]:
# First question and AI response
ai_msg_1 = rag_chain.invoke({"input": question, "chat_history": chat_history.get(id, [])})
add_to_chat_history(id, question, ai_msg_1["answer"])

In [ ]:
print(ai_msg_1["answer"])

second_question = f"Produce a script for this mapping. Include more complex attributes of the {ENTITY} schema, like arrays if applicable. Return only script json object which contains body and inputSrcPaths inside script attribute."


In [ ]:
# Second question and AI response
second_question = f"Write a JavaScript Nashorn script to map the given input to given {ENTITY}. Include more complex attributes of the {ENTITY} schema, like arrays if applicable. Check in the instruction how to write inputSrcPaths for nested objects. Return only script json object which contains body and inputSrcPaths inside script attribute."
ai_msg_2 = rag_chain.invoke({"input": second_question, "chat_history": chat_history.get(id, [])})
add_to_chat_history(id, second_question, ai_msg_2["answer"])
print(ai_msg_2["answer"])

In [ ]:
# Second question and AI response
second_question = f"Can you show me the place in the instruction which explains inputSrcPaths for nested objects"
ai_msg_2 = rag_chain.invoke({"input": second_question, "chat_history": chat_history.get(id, [])})
add_to_chat_history(id, second_question, ai_msg_2["answer"])
print(ai_msg_2["answer"])

In [ ]:
print(chat_history)

In [ ]:
clear_chat_history(id)

In [ ]:
print(ai_msg_1["context"][0])
